In [ ]:
#%cd G:\Meine Ablage\TechLabs\Scam Busters
%cd '/Users/nancynyambura/Desktop/Digital Shaper Program'

In [ ]:
# imports
import requests
import json
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import inspect

import datetime as dt
features_id = dt.date.today().strftime('%Y-%m-%d')

# settings to load data
mypath = '/Users/nancynyambura/Desktop/Digital Shaper Program/ScrumBusters/Frame 1/Telegram/data/04_raw_financial/'

onlyfiles = [mypath + f for f in listdir(mypath) if (isfile(join(mypath, f)))]

onlyfiles = [f for f in onlyfiles if 'simplified' in f ]


In [ ]:
# set threshold value for the pump indicator (max return within 1 minute)
successful_pump_threshold = 1.1
time_to_pump = 5
expected_pump_row = -31

In [ ]:
pump_status_dict = {
    0:'failed',
    1:'success',
    2:'maybe - wrong time',
    3:'maybe - low initial return',
    4:'maybe - different reason'
}

import pickle
with open('pump_status_dictionary.pkl', 'wb') as f:
    pickle.dump(pump_status_dict, f)

In [ ]:
def get_pump_success(indicator, indicator_idx, df_length):
    if (indicator >= successful_pump_threshold) & (indicator_idx == df_length+expected_pump_row):
        return 1
    if (indicator >= successful_pump_threshold):
        return 2
    else:
        return 0

def pump_success_indicator(file):
    df = pd.read_csv(file)
    high, open = df.high, df.open
    idx_indicator = (high/open).idxmax()
    indicator = (high/open)[idx_indicator]
    high_max=df['high'].rolling(2).max()
    open_max=df['open'].rolling(2).max()
    open_max = open_max.dropna().reset_index(drop=True)
    max_possible_return = df.iloc[-40:-25].high.max() / df.iloc[-40:-25].low.min()
    max_5m_return = (high_max/open_max).dropna().max()
    max_5m_return_idx = (high_max/open_max).dropna().idxmax()
    pump_success = get_pump_success(indicator,idx_indicator,len(df))
    return {'indicator':indicator, 'idx_indicator':idx_indicator,
        'max_possible_return':max_possible_return, 'pump_success':pump_success}

all_indicators = pd.DataFrame([
    pump_success_indicator(file)
    for file
    in onlyfiles
])

In [ ]:
all_indicators

In [ ]:
id = 0
raw_df = pd.read_csv(onlyfiles[id])
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.suptitle('high, volume')
fig.set_size_inches(18.5, 6.5)
ax1.plot(raw_df['Unnamed: 0'], raw_df.high)
ax2.plot(raw_df['Unnamed: 0'], raw_df.volume)

In [ ]:
def returns_before_pump(df,hours_to_pump):
    '''returns the ROI if investing hours_to_pump 
    number of hours before the exit and exiting
    time_to_pump minutes before the pump'''
    roi = df.high.iloc[-1] / df.high.iloc[ - hours_to_pump * 60 - 1 ]
    return roi

def get_btc_volume(df,hours_to_pump):
    '''returns the total volume for t hours 
    up until x minutes before the pump'''
    return df.quote_asset_volume.iloc[-hours_to_pump*60-1:].sum()

def get_coin_volume(df,hours_to_pump):
    return df.volume.iloc[-hours_to_pump*60-1:].sum()

def get_coin_volume_avg(df,hours_to_pump):
    return df.volume.iloc[-hours_to_pump*60-1:].mean()

def get_coin_volume_std(df,hours_to_pump):
    return df.volume.iloc[-hours_to_pump*60-1:].std()

def get_coin_volume_hourly(df,hours_to_pump):
        return df.volume.iloc[-hours_to_pump*60-1:].sum()

def get_trades_std(df,hours_to_pump):
        return df.n_trades.iloc[-hours_to_pump*60-1:].std()


def get_log_return(df,hours_to_pump):
    return 1

In [ ]:
# full engineering for a single df
def compute_features(df):
    # cut off everything that will be unknown in reality
    df_cutoff = df.iloc[:expected_pump_row-time_to_pump]
    df_to_1h = df.iloc[:expected_pump_row-60]

    # instantiate empty dict
    features = {}

    # get features which need a timespan to make sense
    hrs = [1,3,12,24,36,48,60,72]
    for h in hrs:
        features[f'return_{h}hrs_to_cutoff'] = returns_before_pump(df_cutoff,h)
        features[f'return_{h}hrs_to_1h'] = returns_before_pump(df_to_1h,h)
        features[f'btc_volume_{h}hrs_to_cutoff'] = get_btc_volume(df_cutoff,h)
        features[f'btc_volume_{h}hrs_to_1h'] = get_btc_volume(df_to_1h,h)
        features[f'coin_volume_{h}hrs_to_cutoff'] = get_coin_volume(df_cutoff,h)
        features[f'coin_volume_{h}hrs_to_1h'] = get_coin_volume(df_to_1h,h)
        features[f'coin_volume_avg_{h}hrs_to_1h'] = get_coin_volume_avg(df_to_1h,h)
        features[f'coin_volume_avg_{h}hrs_to_cutoff'] = get_coin_volume_avg(df_cutoff,h)
        features[f'coin_volume_std_{h}hrs_to_1h'] = get_coin_volume_std(df_to_1h,h)
        features[f'coin_volume_std_{h}hrs_to_cutoff'] = get_coin_volume_std(df_cutoff,h)
        features[f'trade_std_{h}hrs_to_1h'] = get_trades_std(df_to_1h,h)
        features[f'trade_std_{h}hrs_to_cutoff'] = get_trades_std(df_cutoff,h)

    ## volume hourly score (maybe quarter hourly)
    features[f'coin_volume_hourly_1hrs_to_1h'] = get_coin_volume_hourly(df_to_1h,1)
    # features to be added
    ## coin value in usd=coin price in USD
    ## return std
    ## pumped_before

        
    return features

In [ ]:
# add the label
# pump column

In [ ]:
# get all features
engineered_df = pd.DataFrame([
    compute_features(pd.read_csv(file))
    for file
    in onlyfiles
])
feature_list = engineered_df.columns

In [ ]:
engineered_df.loc[:,'pumped'] = all_indicators.pump_success.replace(2,0)

In [ ]:
# add symbols to the engineered_df

def get_symbol(path_str):
    file_str=path_str[path_str.find('pump')+5:]
    file_str = file_str[:file_str.find('_')]
    return file_str.upper()

In [ ]:
engineered_df.loc[:,'symbol'] = [
    get_symbol(path_str)
    for path_str in onlyfiles
]

In [ ]:
# get market cap data

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '541aa958-c55e-4527-a3b4-5734f576b6bf',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

# unpack and simplify data
market_cap_df = pd.DataFrame(response.json()['data'])
unpacked = pd.DataFrame(market_cap_df.quote.tolist())
unpacked = pd.DataFrame(unpacked.USD.tolist())
market_cap_df.loc[:,'market_cap'] = unpacked.market_cap
market_cap_df = market_cap_df[['symbol','market_cap','date_added']]

In [ ]:
# compute the coin age
market_cap_df.date_added = [
    pd.to_datetime(date).tz_localize(None)
    for date in market_cap_df.date_added
]
today = pd.to_datetime(dt.date.today())
market_cap_df.loc[:,'coin_age'] = [
    (today-date).days/365
    for date in market_cap_df.date_added
]
# keep only relevant data
market_cap_df = market_cap_df[['symbol','market_cap','coin_age']]

In [ ]:
# add market cap
engineered_df = pd.merge(engineered_df, market_cap_df, on='symbol', how = 'left')#.drop(['Unnamed: 0', 'symbol'], axis = 1)
engineered_df.market_cap = engineered_df.market_cap.fillna(engineered_df.market_cap.min())
# impute missing values
engineered_df.coin_age = engineered_df.coin_age.fillna(engineered_df.coin_age.min())

In [ ]:
engineered_df.to_csv(f'/Users/nancynyambura/Desktop/Digital Shaper Program/ScrumBusters/Frame 1/Telegram/data/05_engineered_features/{features_id}.csv')

In [ ]:
engineered_df.isna().sum()

In [ ]:
engineered_df.columns

In [ ]:
engineered_df.pumped.sum()